## The sixth Lab-assignment (08/02/2022 11:59'AM' - 08/05/2022 11:59PM, 50 points in total)

The purpose of this exercise is to build a simple predicition model which can helpyou understand the workflow of machine learning. 

### Q1 Task Decription (50 Points)

The goal of this assignment is to predict bike share use, given the hour, day, and information about the weather. Companies like Divvy try to predict how much demand there will be for bikes on any given day to allocate resources to redistribute bikes so that, ideally, very few bike stations are ever full (when you can’t park your bike) or empty (when you can’t pick up a bike if you want to).

The data (link: https://github.com/suthapalliuday/INFO5502-Summer2022/tree/main/datasets/lab_assignment_06) in Github provides detailed information on the data set and necessary downloads.

### Data Description

You are provided hourly rental data spanning two years (link: https://github.com/suthapalliuday/INFO5502-Summer2022/tree/main/datasets/lab_assignment_06). For this task, the training set is comprised of the first 16 days of each month, while the test set is the 17-19th day of the month. You must predict the total count of bikes rented during each hour covered by the test set, using only information available prior to the rental period. That is, predict "count" without using "count" or its components "casual" and "registered".

Data Fields

datetime - hourly date + timestamp

season - 1 = spring, 2 = summer, 3 = fall, 4 = winter

holiday - whether the day is considered a holiday

workingday - whether the day is neither a weekend nor holiday

weather -

1 - Clear, Few clouds, Partly cloudy, Partly cloudy

2 - Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist

3 - Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds

4 - Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog

temp - temperature in Celsius

atemp - "feels like" temperature in Celsius

humidity - relative humidity

windspeed - wind speed

casual - number of non-registered user rentals initiated

registered - number of registered user rentals initiated

count - number of total rentals

### Output

Submission Format

Your output (a separate file) must have a header line and should be structured in the following format:

 datetime,count
 
 2011-01-20 00:00:00,0 
 
 2011-01-20 01:00:00,0
 
 2011-01-20 02:00:00,0
 
 ...
 
The tutorial code should demonstrate how to generate such a file from a very simple prediction model. Note, these prediction are to be done on the test file under the data tab, where you do not know the actual count, and should match the rows of the test file in count and order.

Your predictions should be compared to the ground truth information (sample_prediction.csv). Score are calculated using Root Mean Squared Error (RMSE).

### Tips

●	Add features: Pick columns/features from the data you already have. Or make a new feature from the ones you have. For example, the tutorial makes ‘hour’ from the datetime stamp, which seemed very useful. How about ‘month’?

●	Model selection: Try different models. Make sure they are regression models rather than classification models. Tip: random forest regression is a good, all around modeling strategy on complicated data sets.

●	Model tuning: Almost all regression models have parameters to tune (“hyperparameters”). E.g. polynomial regression has the degree of the polynomial (n = 1 for a line, n=2 for a quadratic fit, n=3 for a cubic fit…). Generally, one extreme makes the model too simple (e.g. a line for a curved set of points) and the other extreme makes the model overfit/be too complex, and usually the right choice is in between. For some models it is obvious what to tune (e.g. k for k nearest neighbors regression) and some don’t need much tuning with defaults that often work well. e.g. try changing the number of trees used in the random forest model!

●	Cross validation: The tutorial has a simple way of separating training and test data, however, there are better ways of splitting training and test data. Look into cross validation techniques, which are more reliable than an arbitrary split of training and test data.

●	Separate models for ...: Notice that count comes from just adding casual riders and registered riders. However, what if these two types of riders acted very differently? It might make sense to make two separate models and just add the results of both models together. This is also true for any subsets of your data that may behave wildly differently (e.g. create a separate model for each season?)

### Your code

In [1]:
import os
os.chdir('C:\\Users\Sasi9\Downloads\INFO5502-Summer2022-main\INFO5502-Summer2022-main\datasets\lab_assignment_06')
os.getcwd()

'C:\\Users\\Sasi9\\Downloads\\INFO5502-Summer2022-main\\INFO5502-Summer2022-main\\datasets\\lab_assignment_06'

In [2]:
### You code here:
from datetime import datetime
import pandas as pd
import numpy as np

df = pd.read_csv("train_luc.csv")
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [3]:
df.dtypes

datetime       object
season          int64
holiday         int64
workingday      int64
weather         int64
temp          float64
atemp         float64
humidity        int64
windspeed     float64
casual          int64
registered      int64
count           int64
dtype: object

In [4]:
# converting datetime column to datetime datatype
df['datetime']=pd.to_datetime(df.datetime)
# we extract hour, day, month values from datetime
df['hour'] = df['datetime'].dt.hour
df['month'] = df['datetime'].dt.month

In [5]:
#data.dtypes
df['season'].value_counts(),df['holiday'].value_counts(),df['workingday'].value_counts(),df['weather'].value_counts()

(4    2303
 2    2301
 3    2301
 1    2269
 Name: season, dtype: int64,
 0    8887
 1     287
 Name: holiday, dtype: int64,
 1    6228
 0    2946
 Name: workingday, dtype: int64,
 1    6105
 2    2333
 3     735
 4       1
 Name: weather, dtype: int64)

In [6]:
df = pd.get_dummies(df, columns=['season','holiday','workingday','weather'])
df.head()

,datetime,temp,atemp,humidity,windspeed,casual,registered,count,hour,month,...,season_3,season_4,holiday_0,holiday_1,workingday_0,workingday_1,weather_1,weather_2,weather_3,weather_4
0,2011-01-01 00:00:00,9.84,14.395,81,0.0,3,13,16,0,1,...,0,0,1,0,1,0,1,0,0,0
1,2011-01-01 01:00:00,9.02,13.635,80,0.0,8,32,40,1,1,...,0,0,1,0,1,0,1,0,0,0
2,2011-01-01 02:00:00,9.02,13.635,80,0.0,5,27,32,2,1,...,0,0,1,0,1,0,1,0,0,0
3,2011-01-01 03:00:00,9.84,14.395,75,0.0,3,10,13,3,1,...,0,0,1,0,1,0,1,0,0,0
4,2011-01-01 04:00:00,9.84,14.395,75,0.0,0,1,1,4,1,...,0,0,1,0,1,0,1,0,0,0


In [7]:
df_Y = df['count']
df_Y.shape

(9174,)

In [8]:
df = df.drop(['datetime','casual','registered','count'],axis=1)

In [9]:
df.shape

(9174, 18)

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df,df_Y,test_size=0.25)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((6880, 18), (2294, 18), (6880,), (2294,))

In [21]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
#from sklearn.metrics import mean_squared_error

# Applying Linear Regression to find the customers count of Rentals
LR_Model = LinearRegression()
LR_Model.fit(X_train, y_train)

LR_Model_Pred = LR_Model.predict(X_test)

# finding mean square error & mean absolute error
print("MSE:", mean_squared_error(y_test, LR_Model_Pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, LR_Model_Pred)))
print("MAE:", mean_absolute_error(y_test, LR_Model_Pred))

# finding R2 Score
print("R2 Score:", r2_score(y_test, LR_Model_Pred))

MSE: 20146.02678319189
RMSE: 141.93669991651873
MAE: 105.15469413385007
R2 Score: 0.36045934922397893


In [22]:
from sklearn.tree import DecisionTreeRegressor
# Using Decision Tree Regressor model to find the count values
DT_Reg_Model = DecisionTreeRegressor(random_state=211)
DT_Reg_Model.fit(X_train, y_train)

DT_Model_Pred = DT_Reg_Model.predict(X_test)

# finding mean square error & mean absolute error
print("MSE:", mean_squared_error(y_test, DT_Model_Pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, DT_Model_Pred)))
print("MAE:", mean_absolute_error(y_test, DT_Model_Pred))

# finding R2 Score
print("R2 Score:", r2_score(y_test, DT_Model_Pred))

MSE: 8383.804489973845
RMSE: 91.56311751995912
MAE: 57.244551002615516
R2 Score: 0.7338540329962129


In [23]:
from sklearn.ensemble import RandomForestRegressor
# using Random Forest Regressor to find the count values
RF_Reg_Model = RandomForestRegressor(random_state = 211)
RF_Reg_Model.fit(X_train, y_train)
RF_Model_Pred = RF_Reg_Model.predict(X_test)

# finding mean square error & mean absolute error
print("MSE:", mean_squared_error(y_test, RF_Model_Pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, RF_Model_Pred)))
print("MAE:", mean_absolute_error(y_test, RF_Model_Pred))

# finding R2 Score
print("R2 Score:", r2_score(y_test, RF_Model_Pred))

MSE: 4475.92936219332
RMSE: 66.9023868198536
MAE: 44.024976795920345
R2 Score: 0.8579105047396803


In [15]:
# load the test dataset
df_test = pd.read_csv("test_luc.csv")

df_test['datetime'] = pd.to_datetime(df_test.datetime)
# we extract the hour and month
df_test['hour'] = df_test['datetime'].dt.hour
df_test['month'] = df_test['datetime'].dt.month

df_test = pd.get_dummies(df_test, columns=['season','holiday','workingday','weather'])
df_test = df_test.drop(['datetime'],axis=1)
df_test['weather_4'] = 0
df_test.head()

#After loading Test data, we have Random Forest Regression model as final model with High R-squared value with 85%
# So I created predictions for test dataset by using Random Forest Regression model
RF_Reg_Model_Predictions = RF_Reg_Model.predict(df_test)
RF_Reg_Model_Predictions

array([ 35.46,  29.27,  26.74, ..., 147.15, 145.05,  92.77])

In [24]:
df_test = pd.read_csv("test_luc.csv")
df_test['count'] = RF_Reg_Model_Predictions
df_test_1 = df_test[['datetime', 'count']]
df_test_1

,datetime,count
0,2011-01-17 00:00:00,35.46
1,2011-01-17 01:00:00,29.27
2,2011-01-17 02:00:00,26.74
3,2011-01-17 03:00:00,13.13
4,2011-01-17 04:00:00,6.79
...,...,...
1707,2012-12-19 19:00:00,343.55
1708,2012-12-19 20:00:00,226.56
1709,2012-12-19 21:00:00,147.15
1710,2012-12-19 22:00:00,145.05


In [25]:
df_test_1.to_csv('Sasi_prediction.csv', index=False, header=True)

 Question 2 (20 points) Build a regession model to predict the change in price of the stock overtime. It can either be linear regression or non-linear regression. You can download the dataset from here: https://github.com/suthapalliuday/INFO5502-Summer2022/blob/main/datasets/portfolio_data.csv. The implementation of the LSTM on this same dataset was posted here as an reference: https://www.kaggle.com/code/faressayah/stock-market-analysis-prediction-using-lstm/notebook

 ### Your code

In [1]:
### You code here:
